In [21]:
import sys
import pickle
import numpy as np
import pandas as pd
import nltk
from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, classification_report
nltk.download('stopwords')
from nltk.corpus import stopwords
import re
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC, LinearSVC

# import gensim
# from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, AdaBoostClassifier
# import matplotlib.pyplot as plt
# from bs4 import BeautifulSoup
# import xgboost
# from sklearn.feature_extraction.text import TfidfTransformer
from time import time
# from fast_ml.model_development import train_valid_test_split
# from scipy.sparse import csr_matrix

In [13]:
# data=pd.read_csv("./Desktop/nlp_a1/train.csv")
data1=pd.read_csv("./Desktop/nlp_a1/train.csv")

In [14]:
punctuation = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
Stopwords = set(stopwords.words('english'))

In [15]:
def preprocess1(txt):
    txt = txt.lower() 
    txt = punctuation.sub(' ', txt)
    text = BAD_SYMBOLS_RE.sub('', txt) 
    txt = ' '.join(word for word in txt.split() if word not in Stopwords) 
    return txt

data1['profile']=data1['profile'].apply(preprocess1)

In [7]:
def tagger(data):
    for i in data.index:
        sent=data['profile'][i]
        tag_sent = pos_tag(word_tokenize(sent))
        ret=[]
        for word,tag in tag_sent:
            if tag==None:
                ret.append(word)
            else:
                ret.append(str(word)+'_'+str(tag))
        data['profile'][i]= ' '.join(ret)
    return data

In [16]:
data1

,profile,profession
0,working children camp community school setting...,teacher
1,holds phd biosystems engineering university co...,professor
2,john served department chair foley’s national ...,attorney
3,20 years experience field neurosurgery. dr. ko...,surgeon
4,years ash built impressive fashion portfolio f...,photographer
...,...,...
127850,clinical interests include facial problems ros...,physician
127851,research interests include development educati...,professor
127852,mrs. susan decostanza marchand's npi number #1...,physician
127853,mr. franklin practices medicine tallahassee fl...,physician


In [17]:
X = data1.profile
y = data1.profession
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state = 1667)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state = 1667)

In [22]:
# Linear Support Vector Classifier

lsvc = Pipeline([('vect', CountVectorizer(ngram_range=(1, 3))),
                ('clf', LinearSVC(verbose=1, class_weight='balanced', C=0.04, max_iter=10000)),
               ])

my_tags=data['profession'].unique()

lsvc.fit(X_train, y_train)
y_pred_train = lsvc.predict(X_train)
print('train accuracy %s' % accuracy_score(y_pred_train, y_train))
y_pred = lsvc.predict(X_val)
print('test accuracy %s' % accuracy_score(y_pred, y_val))
print(classification_report(y_val, y_pred,target_names=my_tags))

[LibLinear]train accuracy 0.9990730190610455
test accuracy 0.8164595463630834
                   precision    recall  f1-score   support

          teacher       0.76      0.69      0.72       163
        professor       0.76      0.64      0.69       304
         attorney       0.88      0.90      0.89       955
          surgeon       0.69      0.67      0.68        82
     photographer       0.82      0.70      0.76        88
          painter       0.74      0.85      0.79       149
     psychologist       0.92      0.97      0.94       381
        filmmaker       0.80      0.82      0.81       115
        physician       0.89      0.79      0.84        42
interior_designer       0.84      0.74      0.79       232
        architect       0.69      0.56      0.62        43
        dietitian       0.63      0.76      0.69       556
           pastor       0.75      0.70      0.73       200
           rapper       0.84      0.79      0.82       555
       journalist       0.81      0.

In [20]:
# Stocastic Gradient Classifier

from sklearn.linear_model import SGDClassifier

sgd = Pipeline([('vect', CountVectorizer(ngram_range=(1, 3))),
                #('tfidf', TfidfTransformer()),
                ('clf', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, max_iter=50, tol=None)),
               ])

a=time()
sgd.fit(X_train, y_train)

y_pred_train = sgd.predict(X_train)
print('train accuracy %s' % accuracy_score(y_pred_train, y_train))
y_pred = sgd.predict(X_test)
print('test accuracy %s' % accuracy_score(y_pred, y_test))
    
b=time()
print(b-a)
my_tags=data['profession'].unique()

print(classification_report(y_test, y_pred,target_names=my_tags))

train accuracy 0.9695351576833201
test accuracy 0.8160488033786955
480.23806858062744
                   precision    recall  f1-score   support

          teacher       0.83      0.72      0.77       196
        professor       0.77      0.60      0.67       329
         attorney       0.84      0.90      0.87      1058
          surgeon       0.92      0.62      0.74        73
     photographer       0.85      0.70      0.77       103
          painter       0.76      0.84      0.79       188
     psychologist       0.88      0.94      0.91       432
        filmmaker       0.84      0.80      0.82       137
        physician       0.82      0.70      0.76        47
interior_designer       0.80      0.77      0.79       227
        architect       0.75      0.46      0.57        46
        dietitian       0.67      0.73      0.70       640
           pastor       0.84      0.78      0.81       231
           rapper       0.88      0.77      0.82       589
       journalist       0.77

In [ ]:
pip freeze > requirements.txt

In [25]:
op=pd.read_csv("./Desktop/nlp_a1/output.csv")
test=pd.read_csv("./Desktop/nlp_a1/test.csv")


In [39]:
test['profile']=test['profile'].apply(preprocess1)
y_pred=lsvc.predict(test.profile)


In [37]:
X_val.type

AttributeError: 'Series' object has no attribute 'type'

In [38]:
lsvc.predict(test.profile)

array(['teacher', 'professor', 'attorney', 'surgeon', 'photographer',
       'professor', 'painter', 'psychologist', 'professor', 'professor',
       'filmmaker', 'physician', 'physician', 'professor',
       'interior_designer', 'physician', 'professor', 'professor',
       'teacher', 'professor', 'professor', 'professor', 'attorney',
       'professor', 'teacher'], dtype=object)

In [40]:
print(classification_report(op, y_pred))

                   precision    recall  f1-score   support

         attorney       1.00      1.00      1.00         2
        filmmaker       1.00      1.00      1.00         1
interior_designer       1.00      1.00      1.00         1
          painter       1.00      1.00      1.00         1
     photographer       1.00      1.00      1.00         1
        physician       1.00      1.00      1.00         3
        professor       1.00      1.00      1.00        11
     psychologist       1.00      1.00      1.00         1
          surgeon       1.00      1.00      1.00         1
          teacher       1.00      1.00      1.00         3

         accuracy                           1.00        25
        macro avg       1.00      1.00      1.00        25
     weighted avg       1.00      1.00      1.00        25



In [41]:
y_pred = lsvc.predict(X_test)
print('test accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred,target_names=my_tags))

test accuracy 0.8165180666353824
                   precision    recall  f1-score   support

          teacher       0.80      0.74      0.77       196
        professor       0.73      0.65      0.69       329
         attorney       0.86      0.90      0.88      1058
          surgeon       0.75      0.68      0.71        73
     photographer       0.84      0.70      0.76       103
          painter       0.76      0.82      0.79       188
     psychologist       0.90      0.94      0.92       432
        filmmaker       0.84      0.81      0.83       137
        physician       0.89      0.72      0.80        47
interior_designer       0.80      0.78      0.79       227
        architect       0.76      0.57      0.65        46
        dietitian       0.64      0.76      0.70       640
           pastor       0.82      0.77      0.79       231
           rapper       0.86      0.80      0.83       589
       journalist       0.72      0.82      0.77       245
          dentist     